In [1]:
from google.cloud import aiplatform

project_id = "no-latency-labs"
region = "us-central1"
staging_bucket="gs://bucket-name"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

job = aiplatform.CustomTrainingJob(
        display_name="bikeshare-training-job",
        script_path="model-training-code.py",
        container_uri="us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest",
        requirements=["gcsfs"]
    )

job.run(
    replica_count=1,
    machine_type="n1-standard-4",
    sync=True
)
job.wait()

Training script copied to:
gs://no-latency-labs-documents/aiplatform-2024-04-26-09:29:12.174-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://no-latency-labs-documents/aiplatform-custom-training-2024-04-26-09:29:12.295 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6382144964599480320?project=499192289487
CustomTrainingJob projects/499192289487/locations/us-central1/trainingPipelines/6382144964599480320 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5469545915498889216?project=499192289487
CustomTrainingJob projects/499192289487/locations/us-central1/trainingPipelines/6382144964599480320 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/499192289487/locations/us-central1/trainingPipelines/6382144964599480320 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob proj

In [2]:
display_name = "bikeshare-model-sdk"
artifact_uri = "gs://nl-datascience/model-artifact"
serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        sync=False
    )

Creating Model
Create Model backing LRO: projects/499192289487/locations/us-central1/models/5759700402639470592/operations/4424151700286210048
Model created. Resource name: projects/499192289487/locations/us-central1/models/5759700402639470592@1
To use this Model in another session:
model = aiplatform.Model('projects/499192289487/locations/us-central1/models/5759700402639470592@1')


In [3]:
deployed_model_display_name = "bikeshare-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

Creating Endpoint
Create Endpoint backing LRO: projects/499192289487/locations/us-central1/endpoints/1847150947361357824/operations/1421376648736931840
Endpoint created. Resource name: projects/499192289487/locations/us-central1/endpoints/1847150947361357824
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/499192289487/locations/us-central1/endpoints/1847150947361357824')
Deploying model to Endpoint : projects/499192289487/locations/us-central1/endpoints/1847150947361357824
Deploy Endpoint model backing LRO: projects/499192289487/locations/us-central1/endpoints/1847150947361357824/operations/5834904283560017920
Endpoint model deployed. Resource name: projects/499192289487/locations/us-central1/endpoints/1847150947361357824


In [ ]:
# endpoint = aiplatform.Endpoint('projects/{project-number}/locations/us-central1/endpoints/{endpoint-id}')

In [4]:
INSTANCE = [0.24, 0.81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

instances_list = [INSTANCE]
prediction = endpoint.predict(instances_list)
print(prediction)

Prediction(predictions=[4.329191250735414], deployed_model_id='2019651127619878912', metadata=None, model_version_id='1', model_resource_name='projects/499192289487/locations/us-central1/models/5759700402639470592', explanations=None)


In [ ]:
gcs_input_uri = 'gs://bucket-name/input-dataset/batch-data.csv'

BUCKET_URI = "gs://bucket-name/bikeshare-batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="bikeshare_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    generate_explanation=True,
    sync=False
)